# Summary

## inputs
This notebook has three inputs:
1. A NFL `scores` dataset having 1 row per game
2. Our NFL `gameplay` dataset - having many rows per game - each row is a 'play' in the games
3. An NFL `teams` dataset that matches team names to the abbreviation used in the gameplay data  (e.g. Green Bay Packers == GB)

The gameplay data does not have a clear final score - it is primarily concerned with the plays themselves - to get the actual score is hit-and-miss.

## goal
* The goal is to clean and enrich the data so that we can join gameplay data to the actual scores for each game

## cleanup
* The gameplay data has one or two incorrect dates
* The scores data has present-day abbreviations for historical games (e.g. Jacksonville was 'JAC' until 2013, then became 'JAX' - in the scores data it's always 'JAX')
* The gameplay data has Jacksonville as 'JAC' all the way to 2016 (should be 2013) - but we are going to leave that for now and conform our scores to that

## outputs
Join-able versions of:
1. A cleaned NFL `scores` dataset having 1 row per game
2. A cleaned NFL `gameplay` dataset - having many rows per game - each row is a 'play' in the games




# 01 - Prepare

## 01.1 - imports

In [74]:
%load_ext autoreload
%autoreload 2

import warnings

import numpy as np
import pandas as pd
import os
import sys

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.float_format', lambda x: '%.5f' % x)

np.random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
module_path = os.path.abspath(os.path.join('../src'))
print("Adding modules", module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

Adding modules /Users/christopherlomeli/Source/courses/datascience/data_science_capstone02/nfl_capstone/src


## 01.2 - setup

In [76]:
RAW_DATA_PATH = '../data/raw'
INTERIM_DATA_PATH='../data/interim'

# inputs
TEAM_SCORES=os.path.join(RAW_DATA_PATH,"spreadspoke_scores.csv")
TEAM_NAMES=os.path.join(RAW_DATA_PATH,"nfl_teams.csv")
GAME_PLAYS=os.path.join(INTERIM_DATA_PATH,"gameplay_facts_cleaned_01.parquet")

# output
CLEAN_FACTS_DF_NAME=os.path.join(INTERIM_DATA_PATH, "gameplay_facts_cleaned_02.parquet")
CLEAN_SCORES_DF_NAME=os.path.join(INTERIM_DATA_PATH, "nfl_scores.parquet")
READ_ME = os.path.join(INTERIM_DATA_PATH,"README.03-cjl-clean.txt")



# 02 - Get scores data

## 02.1 - load spreadscores

In [77]:
# load
scores_df = pd.read_csv(TEAM_SCORES, parse_dates=['schedule_date'])

# clean up column names and data that we'll join on later
scores_df.drop(columns=['team_favorite_id', 'spread_favorite', 'over_under_line', 'weather_detail'], inplace=True)
scores_df['team_away'] = scores_df['team_away'].str.strip()
scores_df['team_home'] = scores_df['team_home'].str.strip()
scores_df.rename(columns={
    'schedule_date': 'date',
    'schedule_season': 'season',
    'schedule_week': 'week',
    'team_home': 'home_team',
    'team_away': 'away_team'
}, inplace=True)

scores_df.dtypes

date                   datetime64[ns]
season                          int64
week                           object
schedule_playoff                 bool
home_team                      object
score_home                      int64
score_away                      int64
away_team                      object
stadium                        object
stadium_neutral                  bool
weather_temperature           float64
weather_wind_mph              float64
weather_humidity              float64
dtype: object

## 02.2 - load teams list

In [78]:
# load
team_df = pd.read_csv(TEAM_NAMES)

# clean up column names and data that we'll join on later
team_df['team_name'] = team_df['team_name'].str.strip()
team_df['team_id'] = team_df['team_id'].str.strip()
team_df.drop(columns=['team_name_short', 'team_id_pfr','team_conference', 'team_conference_pre2002', 'team_division', 'team_division_pre2002'], inplace=True)

team_df.head()

,team_name,team_id
0,Arizona Cardinals,ARI
1,Atlanta Falcons,ATL
2,Baltimore Colts,IND
3,Baltimore Ravens,BAL
4,Boston Patriots,NE


## 02.3 - merge team_ids into score_df

In [79]:
def merge_team_id(scores_df, home_or_away_team):
    id_name = home_or_away_team.strip() + "_id"
    df2 = scores_df.merge(team_df, left_on=home_or_away_team, right_on='team_name', how='left', indicator=True)
    df2.loc[(df2.season < 2016) & (df2['team_id'] == 'JAX'), 'team_id'] = 'JAC'
    df2.rename(columns={'team_id': id_name}, inplace=True)
    cf = df2.loc[( df2[home_or_away_team] != df2.team_name), [home_or_away_team]].sum().item()
    assert cf == 0
    df2.drop(columns=['_merge', 'team_name'], inplace=True)
    return df2

In [80]:
print("scores now has a home_team_id (abbreviation)")
scores_df = merge_team_id(scores_df=scores_df, home_or_away_team='home_team')
scores_df[['home_team', 'home_team_id']].head()

scores now has a home_team_id (abbreviation)


,home_team,home_team_id
0,Miami Dolphins,MIA
1,Houston Oilers,TEN
2,San Diego Chargers,SD
3,Miami Dolphins,MIA
4,Green Bay Packers,GB


In [81]:
print("scores now has a away_team_id (abbreviation)")
scores_df = merge_team_id(scores_df=scores_df, home_or_away_team='away_team')
scores_df[['away_team', 'away_team_id']].head()

scores now has a away_team_id (abbreviation)


,away_team,away_team_id
0,Oakland Raiders,OAK
1,Denver Broncos,DEN
2,Buffalo Bills,BUF
3,New York Jets,NYJ
4,Baltimore Colts,IND


# 03 - get gameplay data

In [82]:
# load
gameplay_df = pd.read_parquet(GAME_PLAYS)

gameplay_df.dtypes

date                               object
game_id                             int64
drive                               int64
qtr                                 int64
down                              float64
time                               object
time_under                          int64
time_secs                         float64
play_time_diff                    float64
sideof_field                       object
yrdln                             float64
yrdline100                        float64
ydstogo                             int64
ydsnet                              int64
goal_to_go                        float64
first_down                        float64
posteam                            object
defensive_team                     object
desc                               object
play_attempted_key                  int64
yards_gained                        int64
sp_key                              int64
touchdown_key                       int64
safety_key                        

In [83]:
gameplay_df.dtypes

date                               object
game_id                             int64
drive                               int64
qtr                                 int64
down                              float64
time                               object
time_under                          int64
time_secs                         float64
play_time_diff                    float64
sideof_field                       object
yrdln                             float64
yrdline100                        float64
ydstogo                             int64
ydsnet                              int64
goal_to_go                        float64
first_down                        float64
posteam                            object
defensive_team                     object
desc                               object
play_attempted_key                  int64
yards_gained                        int64
sp_key                              int64
touchdown_key                       int64
safety_key                        

# 04 - conform score and gameplay data for joins
Look at the Detroit Lions 2017 season - both datasets should have 16 games

#### 03.1 - check gameplay dataset 2017 Detroit Lions season

In [84]:
gameplay_df.loc[(gameplay_df.season == 2017) & ((gameplay_df.home_team=='DET') | (gameplay_df.away_team=='DET')) , ['season', 'date', 'game_id','home_team', 'away_team']].groupby(['season', 'date', 'game_id','home_team', 'away_team']).count().sort_values(by='date')

Empty DataFrame
Columns: []
Index: [(2017, 2017-09-10, 2017091004, DET, ARI), (2017, 2017-09-18, 2017091800, NYG, DET), (2017, 2017-09-24, 2017092404, DET, ATL), (2017, 2017-10-01, 2017100106, MIN, DET), (2017, 2017-10-08, 2017100802, DET, CAR), (2017, 2017-10-15, 2017101504, NO, DET), (2017, 2017-10-29, 2017102910, DET, PIT), (2017, 2017-11-06, 2017110600, GB, DET), (2017, 2017-11-12, 2017111202, DET, CLE), (2017, 2017-11-19, 2017111900, CHI, DET), (2017, 2017-11-23, 2017112300, DET, MIN), (2017, 2017-12-03, 2017120301, BAL, DET), (2017, 2017-12-10, 2017121007, TB, DET), (2017, 2017-12-16, 2017121600, DET, CHI), (2017, 2017-12-24, 2017122402, CIN, DET), (2017, 2017-12-31, 2017123102, DET, GB)]

#### 03.2 - check scores dataset 2017 Detroit Lions season

In [85]:
scores_df.loc[(scores_df.season==2017) & ((scores_df.home_team=='Detroit Lions') | (scores_df.away_team=='Detroit Lions')), ['season', 'date', 'home_team_id', 'away_team_id']].sort_values(by='date')

,season,date,home_team_id,away_team_id
11883,2017,2017-09-10,DET,ARI
11907,2017,2017-09-18,NYG,DET
11912,2017,2017-09-24,DET,ATL
11934,2017,2017-10-01,MIN,DET
11944,2017,2017-10-08,DET,CAR
11963,2017,2017-10-15,NO,DET
11987,2017,2017-10-29,DET,PIT
12008,2017,2017-11-06,GB,DET
12014,2017,2017-11-12,DET,CLE
12024,2017,2017-11-19,CHI,DET


## 03.3 - conform game_ids as the join key

#### 03.3(a) - fix : the gameplay data has an incorrect date for the 2014 buffalo jets game

In [86]:
# gameplay has the wrong date for the 2014 buffalo jets game
gameplay_df.loc[(gameplay_df['season'] == 2014) &(gameplay_df['home_team'] == 'BUF') &(gameplay_df['away_team'] == 'NYJ'), 'date' ] = '2014-11-24'

In [87]:
# the 2016 jacksonville jaguars game still has them as 'JAC' in the gameplay data
scores_df.loc[(scores_df.date == '2016-09-11') & (scores_df.home_team == 'Jacksonville Jaguars'), 'home_team_id'] =  'JAC'

In [88]:
# the gameplay data date is still an object, and that's good for concatenating later, but we also want a real date
gameplay_df['date_string'] = gameplay_df['date']
gameplay_df['date'] = pd.to_datetime(gameplay_df['date'])

In [89]:
# todo - the gameplay data has Jacksonville as 'JAC' up to 2016 - but it was actually changed to JAX in 2013

### 03.3(b) - add a conformed game_id to each dataset

In [90]:
scores_df['game_id'] = scores_df['date'].astype('string').str.replace("-","")+scores_df.home_team_id.str.lower() + scores_df.away_team_id.str.lower()

In [91]:
gameplay_df['game_id']  = gameplay_df['date_string'].astype('string').str.replace("-","")+gameplay_df.home_team.str.lower() + gameplay_df.away_team.str.lower()

## 03.4 - test the join - iterate and clean until it's 100%

In [92]:
# try a test merge
test_df = gameplay_df.merge(scores_df, left_on='game_id', right_on='game_id', how='left', indicator=True)

In [93]:
# list any failures
m = test_df.loc[(test_df._merge == 'left_only'), ['date_x', 'game_id','home_team_x', 'away_team_x']]
np.unique(m['date_x'].astype('string')+m['home_team_x']+'-'+m['away_team_x'])

array([], dtype=object)

In [94]:
# validate the merge
assert test_df.loc[test_df._merge == 'left_only'].size == 0
assert test_df.loc[test_df._merge == 'right_only'].size == 0

# 05 - output gameplay and score data

In [95]:
gameplay_df.to_parquet(CLEAN_FACTS_DF_NAME, engine='fastparquet',  compression='snappy')
scores_df.to_parquet(CLEAN_SCORES_DF_NAME, engine='fastparquet',  compression='snappy')


In [96]:
with open(READ_ME, 'w') as f:
    f.write(f"\n{os.path.basename(CLEAN_FACTS_DF_NAME)}\ta clean version of gameplay that can be joined to the scores data")
    f.write(f"\n{os.path.basename(CLEAN_SCORES_DF_NAME)}\ta clean version of the scores data that can be joined to gameplay")
